In [1]:
import sqlite3
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
import numpy as np
import pandas as pd

In [3]:
# code to clean up calls.csv and move it to a SQL table

calls = pd.read_csv("data/Call_Data.csv")

def callstime(df, col_arrive):
    yearmonthday = df[col_arrive].astype(str).str.split()
    df["year"] = yearmonthday.apply(lambda x: x[2])
    df["day"] = yearmonthday.apply(lambda x: x[1])
    month_dict = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec":12}
    df["month"] = yearmonthday.apply(lambda x: month_dict[x[0]])
    df["time"] = yearmonthday.apply(lambda x: x[3])
    return df

calls = callstime(calls,"Arrived Time")
calls17 = calls[calls["year"].isin(["2014", "2015", "2016", "2017", "2018", "2019"])]

# calculate and add column for epoch time
bar = pd.to_datetime( calls17["Arrived Time"], format="%b %d %Y %I:%M:%S:%f%p" )
epoch_second2 = bar.map(lambda x: x.value/1e9)
calls17["epoch_calls"] = epoch_second2

rs = calls17["CAD Event Number"].value_counts()
two = []
for v, i in zip(rs.values, rs.index):
    if v >1:
        two.append(i)
# len(two) == 362  
st = [x > 1 for x in rs]
dt =rs[st]

t = calls17[calls17["CAD Event Number"].isin(two)]
indt = t[t["Event Clearance Description"] == "-"].index
indt

calls17.drop(indt, axis=0, inplace=True)

#remove repetitive columns
calls_tosql = calls17.drop(columns =["day", "month", "time", "Precinct", "Sector", "Arrived Time"])
calls_tosql.set_index("epoch_calls", inplace=True)

# create engine
cnx = sqlite3.connect('calls.sqlite')

# create cursor
cur = cnx.cursor()

# move calls to sqlite table - DONT REPEAT
calls_tosql.to_sql("log", con=cnx, index=True, if_exists="replace")


/Users/sarahburgart/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/sarahburgart/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/Users/sarahburgart/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [4]:
len(calls_tosql)


2079514